In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
import pandas as pd
import os
import matplotlib.pyplot as plt


In [3]:
dataname = pd.read_csv('/content/sample_data/california_housing_test.csv')

In [4]:
dataname

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0
...,...,...,...,...,...,...,...,...,...
2995,-119.86,34.42,23.0,1450.0,642.0,1258.0,607.0,1.1790,225000.0
2996,-118.14,34.06,27.0,5257.0,1082.0,3496.0,1036.0,3.3906,237200.0
2997,-119.70,36.30,10.0,956.0,201.0,693.0,220.0,2.2895,62000.0
2998,-117.12,34.10,40.0,96.0,14.0,46.0,14.0,3.2708,162500.0


In [ ]:
path='/content/gdrive/MyDrive/용접_데이터셋/201020_TestData/'
excel_original=pd.DataFrame()
for list in os.listdir(path):
  excel_original=pd.concat([excel_original,pd.read_excel(path+list)],axis=0)

In [ ]:
#excel_original: 모든 파일 통합 데이터
excel=excel_original.drop(['Unnamed: 6','Unnamed: 14','Unnamed: 19','용접선 이탈 (벽)','용접선 이탈'],axis=1)#고려필요 없음(read_me)
excel=excel.drop(['용접속도','송급속도','전압보정','위빙(hz)','위빙(좌)','위빙(우)'],axis=1)#고려필요 없음(read_me) 질문필요
excel=excel.drop(excel[(excel['Start']==1.0)|(excel['End']==1.0)].index)#Start, End에 해당=> 제거(read_me)
excel=excel.drop(['Start','End'],axis=1)
excel=excel.drop(['Pass','형상','두께(1)','두께(2)'],axis=1)#다 일정한 값을 가짐
excel=excel.drop(['WIRESPEED','WINDSPEED','WINDANGLE'],axis=1)#read_me 조건에서 처리 질문필요

In [ ]:
print(excel.isnull().sum())
print(sum(excel.isnull().sum(axis=1)==4))

WELDTIME        0
CURRENT         0
VOLTAGE         0
GAS             0
정상          11816
언더컷         26206
기공          31863
용락          25704
오버랩         31863
dtype: int64
31863


In [ ]:
excel

,WELDTIME,CURRENT,VOLTAGE,GAS,정상,언더컷,기공,용락,오버랩
304,3.05,452.5,30.5,29.9,1.0,NaN,NaN,NaN,NaN
305,3.06,274.2,30.9,29.9,1.0,NaN,NaN,NaN,NaN
306,3.07,328.9,30.3,29.9,NaN,NaN,NaN,1.0,NaN
307,3.08,313.7,30.1,29.9,NaN,NaN,NaN,1.0,NaN
308,3.09,311.7,30.1,29.8,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...
3462,37.41,468.1,29.0,23.2,1.0,NaN,NaN,NaN,NaN
3463,37.42,370.3,29.5,23.2,1.0,NaN,NaN,NaN,NaN
3464,37.43,392.0,29.4,23.2,1.0,NaN,NaN,NaN,NaN
3465,37.44,409.5,29.1,23.2,1.0,NaN,NaN,NaN,NaN


In [ ]:
conditionlist = [
  (excel['정상'] ==1) ,
  (excel['용락'] ==1) ,
  (excel['오버랩'] ==1) ]
choicelist = [1, 2, 3]
excel['label'] = np.select(conditionlist, choicelist, default=-1)

In [ ]:
excel=excel.drop(['정상','언더컷','기공','용락','오버랩'],axis=1)
excel

,WELDTIME,CURRENT,VOLTAGE,GAS,label
304,3.05,452.5,30.5,29.9,1
305,3.06,274.2,30.9,29.9,1
306,3.07,328.9,30.3,29.9,2
307,3.08,313.7,30.1,29.9,2
308,3.09,311.7,30.1,29.8,2
...,...,...,...,...,...
3462,37.41,468.1,29.0,23.2,1
3463,37.42,370.3,29.5,23.2,1
3464,37.43,392.0,29.4,23.2,1
3465,37.44,409.5,29.1,23.2,1


In [ ]:
excel.to_csv('excel.csv',index=False)

In [ ]:
#KNN

# 데이터셋 분리
X = excel.iloc[:, :-1].values
y = excel.iloc[:, 4].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#데이터 표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#훈련 및 예측
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=100)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("정확도 : ", accuracy)

정확도 :  0.6715832418013494


In [ ]:
#의사결정트리
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)
print("정확도: ", tree.score(X_test, y_test))

# 트리의 깊이 제한 max_depth=n
tree = DecisionTreeClassifier(max_depth=7, random_state=0)
tree.fit(X_train, y_train)
print("\n정확도: ", tree.score(X_test, y_test))

정확도:  0.603439618378107

정확도:  0.6867938739643484


In [ ]:
#랜덤포레스트
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=500, random_state=0)
forest.fit(X_train, y_train)

print("정확도: ",format(forest.score(X_test, y_test)))

정확도:  0.6756213909113733


In [ ]:
#로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression

Logistic = LogisticRegression(random_state=0)
Logistic.fit(X_train, y_train)

y_pred = Logistic.predict(X_test)

print('정확도 : ', accuracy_score(y_test, y_pred))

정확도 :  0.6349485312578459


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#XGB

from xgboost import plot_importance
from xgboost import XGBRegressor

model = XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)

pred = model.predict(X_test)
print(pred)

[1.2385575  0.94379205 0.89110196 ... 0.6333338  0.8089417  0.48997694]


In [ ]:
model = XGBRegressor(n_estimators=500, learning_rate=0.1)
model.fit(X_train, y_train, verbose=False)

pred = model.predict(X_test)
print(pred)

[17:19:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[1.2191459  0.92832875 0.77061796 ... 0.6709341  0.7856158  0.727394  ]


In [ ]:
# lgbm
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators=500, 
                      num_leaves=150, 
                      subsample=0.5,
                      min_child_samples=3, 
                      max_depth=4)

lgbm.fit(X_train,y_train)
y_pred = lgbm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도 : ", accuracy)

정확도 :  0.6995982927441627


In [ ]:
# lightgbm for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
# define dataset
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, random_state=1)
# evaluate the model
model = LGBMRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# fit the model on the whole dataset
model = LGBMRegressor()
model.fit(X, y)
# make a single prediction
row = [[2.02220122, 0.31563495, 0.82797464, -0.30620401, 0.16003707, -1.44411381, 0.87616892, -0.50446586, 0.23009474, 0.76201118]]
yhat = model.predict(row)
print('Prediction: %.3f' % yhat[0])

MAE: -12.739 (1.408)
Prediction: -82.040
